In [12]:
import requests
import wget
import pandas as pd
import glob
import geoip2.database
# from geolite2 import geolite2

# Downloading files from Rapid 7:

In [2]:
headers = {'X-Api-Key' : 'paste your API key here'}
base_url='https://us.api.insight.rapid7.com/opendata/studies/'

In [6]:
tcp_response = requests.get(url=base_url+'sonar.tcp/', headers=headers)

In [28]:
tcp_response_json = tcp_response.json()
tcp_filelist = tcp_response_json['sonarfile_set']
telnet_filelist = [x for x in tcp_filelist if 'telnet_23' in x and '2018' in x and 'telnet_2323' not in x ]
print(telnet_filelist)
print(len(telnet_filelist))

['2018-12-25-1545700211-telnet_23.csv.gz', '2018-12-18-1545095091-telnet_23.csv.gz', '2018-12-11-1544490291-telnet_23.csv.gz', '2018-12-04-1543885489-telnet_23.csv.gz', '2018-11-28-1543420788-telnet_23.csv.gz', '2018-11-20-1542675891-telnet_23.csv.gz', '2018-11-13-1542071123-telnet_23.csv.gz', '2018-11-06-1541468335-telnet_23.csv.gz', '2018-10-30-1540861786-telnet_23.csv.gz', '2018-10-23-1540265080-telnet_23.csv.gz', '2018-10-16-1539651867-telnet_23.csv.gz', '2018-10-09-1539047007-telnet_23.csv.gz', '2018-10-03-1538554769-telnet_23.csv.gz', '2018-09-25-1537859705-telnet_23.csv.gz', '2018-09-18-1537232683-telnet_23.csv.gz', '2018-09-11-1536627805-telnet_23.csv.gz', '2018-09-04-1536032411-telnet_23.csv.gz', '2018-08-29-1535529534-telnet_23.csv.gz', '2018-08-22-1534928946-telnet_23.csv.gz', '2018-08-15-1534352629-telnet_23.csv.gz', '2018-08-13-1534169173-telnet_23.csv.gz', '2018-08-13-1534164000-telnet_23.csv.gz', '2018-08-08-1533713652-telnet_23.csv.gz', '2018-08-06-1533578563-telnet_23.

In [9]:
udp_response = requests.get(url=base_url+'sonar.udp/', headers=headers)

In [29]:
udp_response_json = udp_response.json()
udp_filelist = udp_response_json['sonarfile_set']
upnp_filelist = [x for x in udp_filelist if 'upnp' in x and '1900' in x and '2018' in x]
print(upnp_filelist)
print(len(upnp_filelist))


['2018-12-03-1543827880-udp_upnp_1900.csv.gz', '2018-11-05-1541431086-udp_upnp_1900.csv.gz', '2018-10-02-1538449908-udp_upnp_1900.csv.gz', '2018-09-27-1538008641-udp_upnp_1900.csv.gz', '2018-09-03-1536013125-udp_upnp_1900.csv.gz', '2018-08-07-1533605769-udp_upnp_1900.csv.gz', '2018-07-02-1530508141-udp_upnp_1900.csv.gz', '2018-06-04-1528088941-udp_upnp_1900.csv.gz', '2018-05-07-1525669741-udp_upnp_1900.csv.gz', '2018-04-02-1522645741-udp_upnp_1900.csv.gz', '2018-03-05-1520226541-udp_upnp_1900.csv.gz', '2018-02-05-1517807341-udp_upnp_1900.csv.gz', '2018-01-01-1514783341-udp_upnp_1900.csv.gz']
13


Project sonar scans the IPv4 more than once a month. To prevent our dataframes from becoming to big, only download the first scans of the month by filling in the correct indices in indices_telnet and indices_upnp. 

After downloading, extract the .gz files and move all the scan results (so <b>NOT</b> the folder inside the archive, but the file in that folder) into Data/Rapid7/Telnet/2018 and Data/Rapid7/UPnP/2018 manually

In [32]:
filelist =   []
response_url = []
study_upnp = 'sonar.udp'
study_telnet = 'sonar.tcp'
location_telnet = './Data/Rapid7/Telnet/'
location_upnp = './Data/Rapid7/UPnP/'

indices_telnet = [3,7]
indices_upnp = [0,1]

# for i in indices_upnp:
#     file = upnp_filelist[i]
#     complete_url = base_url + study_upnp + '/' + file + '/download/'
# #     print(complete_url)
#     response = requests.get(url = complete_url, headers=headers)
#     response_url.append(response.json()['url'])
#     wget.download(response.json()['url'],location_upnp+upnp_filelist[i])
    
for i in indices_telnet:
    file = telnet_filelist[i]
    complete_url = base_url + study_telnet + '/' + file + '/download/'
#     print(complete_url)
    response = requests.get(url = complete_url, headers=headers)
    response_url.append(response.json()['url'])
    wget.download(response.json()['url'],location_telnet+telnet_filelist[i])
    

100% [........................................................................] 84691240 / 84691240

# Adding the Country for each IP address:

### This will take a long time!

In [36]:
#THIS TAKES A LONG TIME!
path = r'.\Data\Rapid7\Telnet\2018' 
all_files = glob.glob(path+"/*")
for f in all_files:
    print(f)
    
df_telnet_2018 = pd.concat((pd.read_csv(f,dtype={"saddr":str,},usecols=['timestamp_ts','saddr'],parse_dates=['timestamp_ts'], date_parser=lambda epoch: pd.to_datetime(epoch, unit='s')) for f in all_files))

.\Data\Rapid7\Telnet\2018\2019-09-19-1568929555-telnet_23


In [34]:
#THIS TAKES A LONG TIME!
path = r'.\Data\Rapid7\UPnP\2018' 
all_files = glob.glob(path+"/*")
for f in all_files:
    print(f)
    
df_upnp_2018 = pd.concat((pd.read_csv(f,dtype={"saddr":str,},usecols=['timestamp_ts','saddr'],parse_dates=['timestamp_ts'], date_parser=lambda epoch: pd.to_datetime(epoch, unit='s')) for f in all_files))

.\Data\Rapid7\UPnP\2018\2018-11-05-1541431086-udp_upnp_1900


In [27]:
def get_country(ip):
    try:
        x = reader.country(ip)
#         print(type(x))
    except Exception:
        return pd.np.nan
#     try:
    return x.country.iso_code
#     except KeyError:
#         return pd.np.nan
        

reader = geoip2.database.Reader('./GeoLite2-Country.mmdb')

In [38]:
unique_ips = df_telnet_2018['saddr'].unique()
# make series out of it
unique_ips = pd.Series(unique_ips, index = unique_ips)
# map IP --> country
%timeit -n 1 -r 1 df_telnet_2018['country'] = df_telnet_2018['saddr'].map(unique_ips.apply(get_country))

df_telnet_2018.to_csv(r'.\Data\Rapid7\Telnet\telnet2018.csv',index=False)

41.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
unique_ips = df_upnp_2018['saddr'].unique()
# make series out of it
unique_ips = pd.Series(unique_ips, index = unique_ips)
# map IP --> country
%timeit -n 1 -r 1 df_upnp_2018['country'] = df_upnp_2018['saddr'].map(unique_ips.apply(get_country))

df_upnp_2018.to_csv(r'.\Data\Rapid7\UPnP\upnp2018.csv',index=False)